In [1]:
# Import python libraries
#
import numpy as np
import importlib
import pickle
import os
import plotly.express as px
import torch
from pprint import pprint
from collections import defaultdict
import pandas as pd

# Go one directory back, because all imports are done 
# relative to the root of the project.
#
if 'change_directory_to_root' not in globals():
    change_directory_to_root = True
    project_root = '..'
    os.chdir(project_root)

# Imports own modules.
#
import scripts.Visualization as Visualization
import scripts.ModelTrainer as ModelTrainer
import scripts.utils as utils
import scripts.Simulation_config as Simulation_config
import scripts.Model as model


No CUDA runtime is found, using CUDA_HOME='/home/molu/miniconda3/envs/xlstm'


In [2]:
# Run the whole simulation
#
importlib.reload(model)
importlib.reload(ModelTrainer)
importlib.reload(Simulation_config)

train_all_models = False
if train_all_models:
    ModelTrainer.ModelTrainer().run()


In [5]:
# Get the simulation results
#

importlib.reload(utils)

all_train_histories = utils.Deserialize.get_training_histories('scripts/outputs/all_train_histories.pkl')
all_trained_models = utils.Deserialize.get_trained_models('scripts/outputs/all_trained_models.pth')


In [7]:
# Print the sumarized simulation results (opionally for latex)
#

# Create a nested dictionary of the results
result_per_config = defaultdict(lambda: defaultdict(lambda: defaultdict(dict)))
for key, value in all_train_histories.items():
    model_types = key[0]
    load_profiles = key[1]
    sim_config = key[2]
    loss = value
    result_per_config[sim_config][model_types][load_profiles] = loss

# Calculate and print median and standard deviation for each config and model_type
for config, result_per_model in result_per_config.items():
    pprint(f'Configuration: {config}')
    print_for_latex = False
    latex_string = ''
    
    for model_type, result_per_profile in result_per_model.items():
        
        # Get al list of all losses of the current config and modeltype
        train_losses, test_RMSE, test_sMAPE  = [], [], []
        for load_profiles, results in result_per_profile.items():
            train_losses.append(np.sqrt(float(results['loss'][-1])))
            test_RMSE.append(np.sqrt(float(results['test_loss'][-1])))
            test_sMAPE.append(float(results['test_sMAPE'][-1]))
        
        assert len(train_losses) == config.nrOfComunities
        assert len(test_RMSE) == config.nrOfComunities
        assert len(test_sMAPE) == config.nrOfComunities
        
        decimal_points_RMSE = 4
        decimal_points_sMAPE = 2
        mean_test_RMSE = f'{np.mean(test_RMSE):.{decimal_points_RMSE}f}'
        mean_test_sMAPE = f'{np.mean(test_sMAPE):.{decimal_points_sMAPE}f}'
        std_dev_test_RMSE = f'{np.std(test_RMSE):.{decimal_points_RMSE}f}'
        std_dev_test_sMAPE = f'{np.std(test_sMAPE):.{decimal_points_sMAPE}f}'
        mean_train_RMSE = f'{np.mean(train_losses):.{decimal_points_RMSE}f}'

        if print_for_latex:
            latex_string += f' & {mean_test_sMAPE}'
        else:            
            # Print the results of the current config and modeltype
            print(f'    Model: {model_type}')
            print(f'      Mean Test RMSE: {mean_test_RMSE}')
            print(f'      Mean Test sMAPE: {mean_test_sMAPE}')
            print(f'      Standard Deviation Test RMSE: {std_dev_test_RMSE}')
            print(f'      Standard Deviation Test sMAPE: {std_dev_test_sMAPE}')
            print(f'      Mean Train RMSE: {mean_train_RMSE}\n')

    if print_for_latex == True:
        print(f'Latex Summary for this Configuration: {latex_string}')


("Configuration: Config_of_one_run(modelSize='MEDIUM', doPretraining=True, "
 'doTransferLearning=True, '
 "aggregation_Count='data/london_loadprofiles_74households_each.pkl', "
 'nrOfComunities=20, trainingHistory=466, modelInputHistory=48, '
 "measurementDelay=24, usedModels=('KNN', 'PersistencePrediction', 'xLSTM', "
 "'LSTM', 'Transformer'), epochs=1)")
    Model: KNN
      Mean Test RMSE: 0.5059
      Mean Test sMAPE: 12.95
      Standard Deviation Test RMSE: 0.0558
      Standard Deviation Test sMAPE: 1.38
      Mean Train RMSE: 0.0000

    Model: PersistencePrediction
      Mean Test RMSE: 3.2903
      Mean Test sMAPE: 200.00
      Standard Deviation Test RMSE: 0.2045
      Standard Deviation Test sMAPE: 0.00
      Mean Train RMSE: 0.0000

    Model: xLSTM
      Mean Test RMSE: 1.2732
      Mean Test sMAPE: 35.24
      Standard Deviation Test RMSE: 0.1264
      Standard Deviation Test sMAPE: 2.37
      Mean Train RMSE: 1.6265

    Model: LSTM
      Mean Test RMSE: 1.9633
      M

In [76]:
# Plot the losses over the training.
# (To see, if the models are trained stable.)
#

# Target config to plot
plotted_config = Simulation_config.configs[0]
print(f"Plotted Config:")
pprint(f"{plotted_config}")
filtered_train_histories = dict(
    (key, value) for key, value in all_train_histories.items() 
    if key[2] == plotted_config
)

# Create a combined list of loss values and corresponding run names
combined_loss_data = []
for run_id, history in filtered_train_histories.items():
    
    # Define the labels of the following graph
    model_type, load_profile, act_config = run_id
    label = (model_type, load_profile, act_config.modelSize)    
    combined_loss_data.extend([(label, epoch + 1, np.sqrt(loss)) for epoch, loss in enumerate(history['loss'])])

# Plot the loss of all items over the training epochs
df = pd.DataFrame(combined_loss_data, columns=['Run', 'Epoch', 'Loss'])
fig = px.line(df, x='Epoch', y='Loss', color='Run', labels={'Loss': 'Training RMSE Loss', 'Epoch': 'Epoch'})
fig.update_layout(title='Training Loss Over Epochs for Different Runs', showlegend=False)
fig.update_yaxes(range=[0, 1])
fig.show()


Plotted Config:
("Config_of_one_run(modelSize='MEDIUM', doPretraining=True, "
 'doTransferLearning=True, '
 "aggregation_Count='data/london_loadprofiles_74households_each.pkl', "
 'nrOfComunities=20, trainingHistory=466, predictionHistory=24, '
 "usedModels=('KNN', 'PersistencePrediction', 'xLSTM', 'LSTM', 'Transformer'), "
 'epochs=100)')


In [7]:
# Evaluate detailed power profiles
#

importlib.reload(Visualization)

myModel = all_trained_models[('xLSTM', 'scripts/outputs/file_0.pkl', Simulation_config.configs[0])]
test_profile = "scripts/outputs/file_0.pkl"
with open(test_profile, 'rb') as f:
    (X, Y, modelAdapter) = pickle.load(f)
plotlyApp = Visualization.PlotlyApp(X, Y, myModel, modelAdapter, None, 'UTC+00:00')
plotlyApp.run(myport=8055)


In [35]:
# Print the model parameter sizes
#

importlib.reload(model)

model_types = ['xLSTM', 'LSTM', 'Transformer']
for model_type in model_types:
    for model_size in ['SMALL', 'MEDIUM', 'LARGE']:

        # Test, if the model is run-able
        num_of_features = 18
        m = model.Model(model_type, model_size, num_of_features)
        x = torch.zeros((7, 24*22, num_of_features))
        m.my_model(x)

        # Print the model's parameter count
        nr_of_parameters = m.get_nr_of_parameters(do_print=False)
        print(f"{model_type} - {model_size} has {nr_of_parameters} parameters.")


xLSTM - SMALL has 19791 parameters.
xLSTM - MEDIUM has 26411 parameters.
xLSTM - LARGE has 97679 parameters.
LSTM - SMALL has 17463 parameters.
LSTM - MEDIUM has 44031 parameters.
LSTM - LARGE has 104831 parameters.
Transformer - SMALL has 24423 parameters.
Transformer - MEDIUM has 50863 parameters.
Transformer - LARGE has 113351 parameters.
